In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

import scipy.optimize as opt

from AppliedMacroPlots import *

In [ ]:
# Calibration follows Ljungqvist and Sargent (2012), p. 395

α = 0.33
δ = 0.2
β = 0.95
g = 0.2
σ = 2    # LS call it γ

In [ ]:
def f(k):
    return k**α

def df_dk(k):
    return α * k**(α-1)

In [ ]:
def RCK_ss(variables, τ_c=0):
    
    (k, c) = variables
    
    y = f(k)
    r = df_dk(k) - δ
    
    EE = β*(1+r) - 1                 # Euler equation
    RC = f(k) + (1-δ)*k - g - c - k  # Resource constraint
    
    return [EE, RC]

In [ ]:
τ_c_0 = 0
τ_c_1 = 0.2

results = opt.fsolve(RCK_ss, (1.5, 0.8), args=(τ_c_0))
print('Initial steady state \t=', results)
k_ss_0 = results[0]
c_ss_0 = results[1]

results = opt.fsolve(RCK_ss, (1.5, 0.8), args=(τ_c_1))
print('Final steady state \t=', results)
k_ss_1 = results[0]
c_ss_1 = results[1]

In [ ]:
def Resource_Constraint(k, c, τ_c):
    
    k_n = f(k) + (1-δ)*k - g - c #(1+τ_c)*c
    
    return k_n

def Interest_Rate(k):
    return df_dk(k) - δ

def Euler_Equation(k, c, τ_c, τ_c_n):
    
    k_n = Resource_Constraint(k, c, τ_c)
    
    if k_n > 0:
        r_n = Interest_Rate(k_n)
        c_n = ( (1+τ_c)/(1+τ_c_n) * β * (1+r_n) )**(1/σ) * c
    else:
        c_n = 0
        
    return c_n

In [ ]:
# Forward equations and convergence criterion

def Path(c_0, k_0, Tτ=20, T=50):
    
    T += 1
    
    k_t = np.zeros(T)
    c_t = np.zeros(T)
    τ_c_t = np.zeros(T)
    
    τ_c_t[:Tτ] = τ_c_0
    τ_c_t[Tτ:] = τ_c_1
    
    k_t[0] = k_0
    c_t[0] = c_0
    
    for t in range(T-1):
        k_t[t+1] = Resource_Constraint(k_t[t], c_t[t], τ_c_t[t])
        if k_t[t+1] > 0:
            c_t[t+1] = Euler_Equation(k_t[t], c_t[t], τ_c_t[t], τ_c_t[t+1])
        else:
            k_t[t+1] = 0
            c_t[t+1] = 0
            
    return k_t, c_t

def Path_crit(c_0, k_0, Tτ=20, T=50):
    
    k_t, c_t = Path(c_0, k_0, Tτ, T)
#     k_star, c_star = SteadyState(params)
    
    ss_diff = np.sqrt((k_t-k_ss_1)**2 + (c_t-c_ss_1)**2)
    
    return np.min(ss_diff) + ss_diff[-1]

In [ ]:
# Find the function minimum, starting from an initial guess
result = opt.minimize(Path_crit, c_ss_0, args=(k_ss_0), method='Powell') # 'Nelder-Mead'
print(result)

c_0 = result.x

In [ ]:
k_t, c_t = Path(c_0, k_ss_0)

plt.plot(k_t, c_t, 'ko-')
plt.plot(k_ss_1, c_ss_1, 'ro')
plt.show()

In [ ]:
plt.plot(k_t, lw=2)
plt.hlines(k_ss_1, 0, 50, linestyles='--', lw=1)
plt.show()

plt.plot(c_t, lw=2)
plt.hlines(c_ss_1, 0, 50, linestyles='--', lw=1)
plt.show()

plt.plot(100*c_t/c_ss_0, lw=2)
plt.show()

![Effects of VAT tax hikes](img/RCK/figure-one.jpg)

Source: [Danninger (2014) What Japan’s economy needs now](https://www.weforum.org/agenda/2014/08/japan-consumption-tax-structural-reform-abenomics/)